In [23]:
from glob import glob
import pandas as pd

# Reading data from our CSV files that are stored inside this repository (single files)

try: 
    single_file_accelerometer = pd.read_csv(
        '../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv')
    single_file_gyroscope = pd.read_csv(
        '../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv')
except FileNotFoundError:
    print("Files that you try to use are not available.")

# Reading all CVS files from data/raw/MetaMotion that we will use as a list later 

# Read all files that have csv extension if inside this repo the compiler finds some files that are not with this extension they 
# will be ignored

files = glob('../../data/raw/MetaMotion/*.csv')

# Extract pieces of the file name e.g. A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv and append 
# it to the data frame

data_path = '../../data/raw/MetaMotion/'

first_file = files[0]

# We will extract three variables from the file name: participant: A, exercise: bench, category of sets(e.g. heavy)

participant = first_file.split('-')[0].replace(data_path,'') # on this way we are updating existing path and getting the participate
exercise = first_file.split('-')[1]
category = first_file.split('-')[2].rstrip('123').rstrip('_MetaWear_2019') # with category we have the number of the set so we need to remove it

data_frame = pd.read_csv(first_file)

data_frame['participant'] = participant # adding new columns inside the data frame
data_frame['exercise'] = exercise
data_frame['category'] = category


# Applying the logic for all data that we defined before  

accelerometer_data_frame = pd.DataFrame() # creating an empty data frame
gyroscope_data_frame = pd.DataFrame()

accelerometer_set = 1
gyroscope_set = 1

for file in files:
    participant = file.split('-')[0].replace(data_path,'') # on this way we are updating existing path and getting the participate
    exercise = file.split('-')[1]
    category = file.split('-')[2].rstrip('123').rstrip('_MetaWear_2019') # with category we have the number of the set so we need to remove it

    data_frame = pd.read_csv(file)

    data_frame['participant'] = participant # adding new columns inside the data frame
    data_frame['exercise'] = exercise
    data_frame['category'] = category

    if 'Accelerometer' in file:
        data_frame['set'] = accelerometer_set 
        accelerometer_set += 1
        accelerometer_data_frame = pd.concat([accelerometer_data_frame,data_frame])
 
    elif 'Gyroscrope' in file:
        data_frame['set'] = gyroscope_set
        gyroscope_set += 1
        gyroscrope_data_frame = pd.concat([gyroscope_data_frame,data_frame])

# --------------------------------------------------------------
# Working with datetimes
# --------------------------------------------------------------


# --------------------------------------------------------------
# Turn into function
# --------------------------------------------------------------


# --------------------------------------------------------------
# Merging datasets
# --------------------------------------------------------------


# --------------------------------------------------------------
# Resample data (frequency conversion)
# --------------------------------------------------------------

# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz


# --------------------------------------------------------------
# Export dataset
# --------------------------------------------------------------


        epoch (ms)             time (01:00)  elapsed (s)  x-axis (g)  \
0    1547221363566  2019-01-11T16:42:43.566         0.00      -0.136   
1    1547221363646  2019-01-11T16:42:43.646         0.08      -0.143   
2    1547221363726  2019-01-11T16:42:43.726         0.16      -0.187   
3    1547221363806  2019-01-11T16:42:43.806         0.24      -0.152   
4    1547221363886  2019-01-11T16:42:43.886         0.32      -0.143   
..             ...                      ...          ...         ...   
165  1547473800195  2019-01-14T14:50:00.195        13.20      -0.085   
166  1547473800275  2019-01-14T14:50:00.275        13.28      -0.070   
167  1547473800355  2019-01-14T14:50:00.355        13.36      -0.082   
168  1547473800435  2019-01-14T14:50:00.435        13.44      -0.091   
169  1547473800515  2019-01-14T14:50:00.515        13.52      -0.084   

     y-axis (g)  z-axis (g) participant exercise category  
0         0.986      -0.053           B      ohp    heavy  
1         0.977